# Ensemble model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")
from ensemble import EnsembleModel

d:\anaconda3\envs\cs4248\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, RobertaTokenizerFast

## Step 1: Load models

### Distilbert

In [4]:
distilbert_name = "jeffnjy/distilbert-base-test"
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_name)
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(distilbert_name)

### Albert

In [5]:
albert_name = "jeffnjy/albert-base-test"
albert_tokenizer = AutoTokenizer.from_pretrained(albert_name)
albert_model = AutoModelForQuestionAnswering.from_pretrained(albert_name)

### Roberta

In [6]:
roberta_name = "jeffnjy/roberta-base-test"
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(roberta_name)
roberta_model = AutoModelForQuestionAnswering.from_pretrained(roberta_name)

### Bert

In [7]:
bert_checkpoint = "../tmp/checkpoint/bert-squad-baseline-model"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_checkpoint, local_files_only=True)
bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_checkpoint, local_files_only=True)

## Step 2: Create ensemble model

The weight of each model can be changed here

In [8]:
model_weights = {
    "bert": 1.0,
    "albert": 1.0,
    "distilbert": 1.0,
    "roberta": 1.0,
}
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [9]:
ensemble_model = EnsembleModel(device=device)
ensemble_model.add_model(model=distilbert_model, tokenizer=distilbert_tokenizer, weight=model_weights["distilbert"])
ensemble_model.add_model(model=albert_model, tokenizer=albert_tokenizer, weight=model_weights["albert"])
ensemble_model.add_model(model=roberta_model, tokenizer=roberta_tokenizer, weight=model_weights["roberta"])
ensemble_model.add_model(model=bert_model, tokenizer=bert_tokenizer, weight=model_weights["bert"])

## Step 3: Perform perdiction

### 3.a Perform perdiction on one example

In [10]:
question = "Which NFL team represented the AFC at Super Bowl 50?"
context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) 
for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football 
Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 
7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the 
league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the 
tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl 
L"), so that the logo could prominently feature the Arabic numerals 50."""

#### Option 1: Weighted sum of softmax probabilities (mode="soft")

In [11]:
ensemble_model.predict(question=question, context=context, mode="soft")

'Denver Broncos'

#### Option 2: Weighted votes (mode="hard")

In [12]:
ensemble_model.predict(question=question, context=context, mode="hard")

'Denver Broncos'

### 3.b Perform perdiction on a json dataset and generate a perdiction json file

In [13]:
inpath = "../dataset/dev-v1.1.json"
outpath = "../tmp/prediction/ensemble_all.json"
ensemble_model.generate_prediction_json(inpath, outpath, mode="soft")

100%|██████████| 10570/10570 [40:32<00:00,  4.34it/s] 
